<h1><b><center>Valor de Imóveis em São Paulo - Abril de 2019</center></b></h1>

---
<font size="2" face="Verdana"> <p>O Estado de **São Paulo** é o estado mais populoso do Brasil, com uma média de 46 milhões de habitantes, sendo sua capital, de mesmo nome, a cidade com a maior concentração dos moradores. Situada na região **Sudoeste** do País, possui uma área territorial em média de **248.219,481 KM²**, segundo uma estimativa do IBGE realizada em 2020.<br><br>
Com clima tropical com invernos mais amenos e verões mais quentes, São Paulo é o principal destino dos brasileiros que buscam novas oportunidades, principalmente de trabalho, principalmente por sua extensão, mas também por comportar as maiores empresas e organizações da américa latina.<br><br>
Com essa alta demanda, encontrar uma moradia acaba se tornando um grande desafio, tanto pela variedade de opções quanto pelo custo de vida. Com base em dados coletados e disponibilizados pela startup [Openi Minded](https://openimob.com/) do mês de Abril de 2019 farei uma análise e contruirei um modelo para predição dos valores de venda de apartamentos.

Por: **Felipe Alves Cardozo**</font>

<p  align=center><br><img src='https://www.flytap.com/-/media/Flytap/new-tap-pages/destinations/south-america/brazil/sao-paulo/sao-paulo-banner-mobile-1024x553.jpg' widht='100%'></p>

# Importação dos Dados


Conforme citado, neste projeto, farei a análise dos dados referentes a venda de apartamentos em São Paulo, especificamente do período de Abril de 2019.

In [26]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score
from xgboost import XGBRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Importando os dados a serem analisados
dataset = pd.read_csv('sao-paulo-properties-april-2019.csv')
dataset.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


# Criando o modelo

In [5]:
# Apagando a string "/São Paulo" da coluna District
dataset1 = dataset.copy()

# Corrigindo o nome do bairro
dataset1['District'] = dataset1['District'].apply(lambda x: x.split('/')[0])

# Removendo as colunas 'Latitude', 'Longitude' e 'Property Type'
dataset1.drop(['Property Type', 'Latitude', 'Longitude'], axis=1, inplace=True)

# Filtrando o dataset para utilizarmos apenas vendas
dataset2 = dataset1[dataset1['Negotiation Type'] == 'sale'].copy()
dataset2.drop(['Negotiation Type'], axis=1, inplace=True)

# Vendo as primeiras entradas
dataset2.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District
4901,732600,1000,74,1,2,1,2,1,0,1,0,Vila Madalena
4902,1990000,2400,164,4,5,2,3,1,1,1,0,Vila Madalena
4903,720000,700,70,2,2,1,1,1,0,1,1,Vila Madalena
4904,1680000,1580,155,3,5,3,2,1,0,1,0,Vila Madalena
4905,1200000,900,56,2,2,1,2,0,1,1,0,Vila Madalena


In [38]:
# Separando as variáveis X e y
X = dataset2.drop('Price', axis=1)
y = dataset2['Price']

# Separando teste e treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Treinando o modelo
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Fazendo a previsão
previsao = model.predict(X_test)

# Avaliação
print(f'r2: {r2_score(y_test, previsao):.2f}%')
print(f'MAE: {mean_absolute_error(y_test, previsao):.2f}')
print(f'MSE: {mean_squared_error(y_test, previsao):.2f}')       

r2: 0.90%
MAE: 103876.94
MSE: 55106333820.26


# Salvando o Modelo

In [49]:
# Salvando o modelo em formato Pkl
import pickle

with open('modelo.pkl', 'wb') as file:
    pickle.dump(model, file)

In [47]:
type(model)

xgboost.sklearn.XGBRegressor

In [40]:
# Salvando o nome das colunas
colunas = X_train.columns.values

with open('colunas.names', 'wb') as file:
    pickle.dump(colunas, file)

# Carregando o Modelo

In [50]:
with open('modelo.pkl', 'rb') as file:
    modelo_importado = pickle.load(file)

In [51]:
type(modelo_importado)

xgboost.sklearn.XGBRegressor